In [7]:
import netCDF4
from mpl_toolkits.basemap import Basemap, pyproj
from pyproj import Proj
from matplotlib import dates
import matplotlib.pyplot as plt
import numpy as np
import datetime
import matplotlib.dates as mdates
import matplotlib.cbook as cbook


In [34]:
dataset = netCDF4.Dataset('../data/netCDF_datafiles/selected_data_023_000.nc')
ncep_hrrr = netCDF4.Dataset('http://thredds-jumbo.unidata.ucar.edu/thredds/dodsC/grib/NCEP/HRRR/CONUS_2p5km/TwoD')

In [35]:
print type(dataset)

<type 'netCDF4._netCDF4.Dataset'>


In [ ]:
print dataset.variables

In [36]:
def lat_lons(open_dap_dataset, proj_key = 'LambertConformal_Projection'):
    xg, yg = np.meshgrid(open_dap_dataset.variables['x'][:]*1000.0, open_dap_dataset.variables['y'][:]*1000.0)
    pnyc = pyproj.Proj(proj = 'lcc', 
                       lat_1 = open_dap_dataset.variables[proj_key].latitude_of_projection_origin,
                       lat_2 = open_dap_dataset.variables[proj_key].latitude_of_projection_origin,
                       lat_0 = open_dap_dataset.variables[proj_key].latitude_of_projection_origin,
                       lon_0 = open_dap_dataset.variables[proj_key].longitude_of_central_meridian )
    return pnyc(xg, yg, inverse = True)

In [37]:
lon, lat = lat_lons(ncep_hrrr)

In [21]:
SGPllcrner_lat = 36.04554
SGPllcrner_lon = -98.158014
SGPupcrner_lat = 36.93360
SGPupcrner_lon = -97.025525

a = np.copy(lon, 'K')
b = np.copy(lat, 'K')

for i in range(len(a)):
    for j in range(len(a[i])):
        value = abs(a[i][j] - SGPllcrner_lon)
        a[i][j] = value

SGPllcrner_lon_index = np.where(a == a.min())
print SGPllcrner_lon_index

for i in range(len(b)):
    for j in range(len(b[i])):
        value = abs(b[i][j] - SGPllcrner_lat)
        b[i][j] = value

SGPllcrner_lat_index = np.where(b == b.min())
print SGPllcrner_lat_index

a = np.copy(lon,'K')
b = np.copy(lat, 'K')

for i in range(len(a)):
    for j in range(len(a[i])):
        value = abs(a[i][j] - SGPupcrner_lon)
        a[i][j] = value
        
SGPupcrner_lon_index = np.where(a == a.min())
print SGPupcrner_lon_index

for i in range(len(b)):
    for j in range(len(b[i])):
        value = abs(b[i][j] - SGPupcrner_lat)
        b[i][j] = value

SGPupcrner_lat_index = np.where(b == b.min())
print SGPupcrner_lat_index

(array([213]), array([965]))
(array([624]), array([504]))
(array([1075]), array([1022]))
(array([725]), array([2051]))


In [39]:
print lat.shape
print lon.shape

(1377, 2145)
(1377, 2145)


In [5]:
selected_data_list = []
timeflt_list = []

In [6]:
selected_data_list.extend(('../data/netCDF_datafiles/selected_data_023_000.nc', \
                           '../data/netCDF_datafiles/selected_data_023_001.nc', \
                           '../data/netCDF_datafiles/selected_data_023_002.nc',\
                           '../data/netCDF_datafiles/selected_data_023_003.nc', \
                           '../data/netCDF_datafiles/selected_data_023_004.nc', \
                           '../data/netCDF_datafiles/selected_data_023_005.nc', \
                           '../data/netCDF_datafiles/selected_data_023_006.nc',\
                           '../data/netCDF_datafiles/selected_data_023_007.nc', \
                           '../data/netCDF_datafiles/selected_data_023_008.nc', \
                           '../data/netCDF_datafiles/selected_data_023_009.nc'))

In [27]:
dset = netCDF4.Dataset('../data/netCDF_datafiles/selected_data_011_007.nc')
units = dset.timestamp[8:-1]
timeflt = float(dset.timestamp[0:4])
dateobj = netCDF4.num2date(timeflt, units)
num = dates.date2num(dateobj)
print num
print dateobj
print dset.timestamp

735801.25
2015-07-22 06:00:00
102.0 :: Hour since 2015-07-18T00:00:00Z


In [31]:
np.set_printoptions(threshold='nan')

In [32]:
print dset.variables['Planetary_boundary_layer_height_surface']
pblh = dset.variables['Planetary_boundary_layer_height_surface'][625]
print pblh


<type 'netCDF4._netCDF4.Variable'>
float32 Planetary_boundary_layer_height_surface(y, x)
    units: Hour since 2015-07-18T00:00:00Z
unlimited dimensions: 
current shape = (1377, 2145)
filling on, default _FillValue of 9.96920996839e+36 used

[305.5 298.0625 274.25 269.75 286.5625 302.8125 310.0625 315.1875 317.75
 321.0 319.75 316.125 311.0625 306.3125 303.1875 300.125 296.0 292.4375
 289.5625 285.8125 277.6875 266.25 257.0625 256.375 264.6875 277.375
 288.4375 293.375 289.1875 283.125 278.4375 275.0 270.875 281.0625 284.8125
 283.125 277.625 272.0625 276.0625 282.6875 282.1875 281.75 280.5 277.6875
 279.6875 278.3125 279.625 280.75 280.375 278.4375 276.625 275.6875 276.5
 278.1875 279.375 279.9375 281.5 284.1875 287.4375 290.75 292.375 291.75
 290.5 290.5625 292.1875 294.0 295.625 297.125 298.3125 298.5625 297.8125
 296.75 295.5625 294.5 293.5625 292.25 291.25 291.0 290.875 289.9375 288.0
 287.375 288.5625 289.5625 288.5625 286.3125 285.0625 281.9375 275.0625
 268.3125 264.125 262.562

In [20]:
dset = netCDF4.Dataset('../data/netCDF_datafiles/selected_data_012_006.nc')
units = dset.timestamp[8:-1]
timeflt = float(dset.timestamp[0:4])
dateobj = netCDF4.num2date(timeflt, units)
num = dates.date2num(dateobj)
print num
print dateobj
print dset.timestamp

735801.25
2015-07-22 06:00:00
102.0 :: Hour since 2015-07-18T00:00:00Z


In [21]:
pblh = dset.variables['Planetary_boundary_layer_height_surface'][213][965]
print pblh

0.0


In [7]:
for i in range(len(selected_data_list)):
    dset = netCDF4.Dataset(selected_data_list[i])
    units = dset.timestamp[8:-1]
    timeflt = float(dset.timestamp[0:4])
    dateobj = netCDF4.num2date(timeflt, units)
    num = dates.date2num(dateobj)
    timeflt_list.append(num)
    print num

735801.458333
735801.5
735801.541667
735801.583333
735801.625
735801.666667
735801.708333
735801.75
735801.791667
735801.833333


In [5]:
print timeflt_list

[735801.4583333334, 735801.5, 735801.5416666666, 735801.5833333334, 735801.625, 735801.6666666666, 735801.7083333334, 735801.75, 735801.7916666666, 735801.8333333334]


In [22]:
dataset = netCDF4.Dataset('../data/netCDF_datafiles/selected_data_023_000.nc')

In [29]:
pblh = dataset.variables["Planetary_boundary_layer_height_surface"]
print pblh.dimensions

(u'y', u'x')


In [10]:
datafile = cbook.get_sample_data('goog.npy')
r = np.load(datafile).view(np.recarray)
print type(r)

<class 'numpy.core.records.recarray'>


In [11]:
fig, ax = plt.subplots()
ax.plot(r.date, r.adj_close)

In [17]:
years    = mdates.YearLocator()   # every year
months   = mdates.MonthLocator()  # every month
yearsFmt = mdates.DateFormatter('%Y')
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)
ax.xaxis.set_minor_locator(months)

In [18]:
datemin = datetime.date(r.date.min().year, 1, 1)
datemax = datetime.date(r.date.max().year+1, 1, 1)
ax.set_xlim(datemin, datemax)

(731581.0, 733408.0)

In [19]:
def price(x): return '$%1.2f'%x
ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
ax.format_ydata = price
ax.grid(True)

In [ ]:
fig.autofmt_xdate()

plt.show()